# Model 1: Kana-Kanji Converter (Optimized)

**Task:** Convert hiragana to kanji with context awareness

**Optimization Strategy:**
- Small architecture + better training = high accuracy
- Knowledge distillation-inspired techniques
- Label smoothing + warmup LR + more data

**Target:** ~3-4MB, 85%+ accuracy

## 1. Setup

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
DRIVE_DIR = '/content/drive/MyDrive/Keyboard-Suggestions-ML-Colab'
MODEL_DIR = f"{DRIVE_DIR}/models/gru_japanese_kana_kanji"
os.makedirs(MODEL_DIR, exist_ok=True)
print(f"✓ Model: {MODEL_DIR}")

In [ ]:
!pip install -q tensorflow keras datasets numpy tqdm

In [ ]:
# ============================================================
# CONFIGURATION - BALANCED SIZE + ACCURACY
# ============================================================

TESTING_MODE = True

if TESTING_MODE:
    NUM_EPOCHS = 8         # More epochs for better learning
    BATCH_SIZE = 256
    MAX_SAMPLES = 400000   # More data
else:
    NUM_EPOCHS = 20
    BATCH_SIZE = 256
    MAX_SAMPLES = 800000

# Balanced model specs (~3-4MB keras, ~2MB TFLite FP16)
CHAR_VOCAB_SIZE = 2500     # Balanced vocab
MAX_SEQ_LENGTH = 25        # Balanced length
EMBEDDING_DIM = 48         # Balanced embedding
GRU_UNITS = 96             # Balanced GRU

# Training optimizations
LABEL_SMOOTHING = 0.1      # Prevents overconfidence
DROPOUT_RATE = 0.15        # Regularization
WARMUP_EPOCHS = 2          # Warmup before decay

SPECIAL_TOKENS = ['<PAD>', '<UNK>', '<BOS>', '<EOS>']

print(f"Config: epochs={NUM_EPOCHS}, samples={MAX_SAMPLES:,}")
print(f"Model: Embed={EMBEDDING_DIM}, GRU={GRU_UNITS}, Vocab={CHAR_VOCAB_SIZE}")

## 2. Load Dataset

In [ ]:
from datasets import load_dataset

print("Loading zenz-v2.5-dataset...")

try:
    dataset = load_dataset(
        "Miwa-Keita/zenz-v2.5-dataset",
        data_files="train_wikipedia.jsonl",
        split=f"train[:{MAX_SAMPLES}]"
    )
except:
    dataset = load_dataset(
        "Miwa-Keita/zenz-v2.5-dataset",
        split=f"train[:{MAX_SAMPLES}]"
    )

print(f"✓ Loaded {len(dataset):,} samples")
print("\nSample data:")
for i in range(3):
    print(f"  {dataset[i]['input'][:25]} → {dataset[i]['output'][:25]}")

## 3. Build Vocabulary

In [ ]:
from collections import Counter
from tqdm import tqdm

print("Building vocabulary...")

char_counts = Counter()
for item in tqdm(dataset, desc="Counting"):
    char_counts.update(list(item.get('input', '')))
    char_counts.update(list(item.get('output', '')))

print(f"✓ Found {len(char_counts):,} unique chars")

char_to_idx = {tok: i for i, tok in enumerate(SPECIAL_TOKENS)}
for char, _ in char_counts.most_common(CHAR_VOCAB_SIZE - len(SPECIAL_TOKENS)):
    char_to_idx[char] = len(char_to_idx)

idx_to_char = {v: k for k, v in char_to_idx.items()}
vocab_size = len(char_to_idx)
print(f"✓ Vocab: {vocab_size:,}")

## 4. Create Training Data

In [ ]:
import numpy as np
import tensorflow as tf

print("Creating training data...")

def encode_seq(text, max_len, add_bos=False, add_eos=False):
    tokens = list(text)
    if add_bos:
        tokens = ['<BOS>'] + tokens
    if add_eos:
        tokens = tokens + ['<EOS>']
    ids = [char_to_idx.get(c, char_to_idx['<UNK>']) for c in tokens]
    if len(ids) < max_len:
        ids = ids + [char_to_idx['<PAD>']] * (max_len - len(ids))
    return ids[:max_len]

encoder_inputs, decoder_inputs, decoder_targets = [], [], []

for item in tqdm(dataset, desc="Processing"):
    kana = item.get('input', '').strip()
    kanji = item.get('output', '').strip()
    
    if not kana or not kanji:
        continue
    if len(kana) > MAX_SEQ_LENGTH - 2 or len(kanji) > MAX_SEQ_LENGTH - 2:
        continue
    
    encoder_inputs.append(encode_seq(kana, MAX_SEQ_LENGTH))
    decoder_inputs.append(encode_seq(kanji, MAX_SEQ_LENGTH, add_bos=True))
    decoder_targets.append(encode_seq(kanji, MAX_SEQ_LENGTH, add_eos=True))

encoder_inputs = np.array(encoder_inputs)
decoder_inputs = np.array(decoder_inputs)
decoder_targets = np.array(decoder_targets)

print(f"\n✓ {len(encoder_inputs):,} training pairs")

In [ ]:
# Shuffle and split
indices = np.random.permutation(len(encoder_inputs))
encoder_inputs = encoder_inputs[indices]
decoder_inputs = decoder_inputs[indices]
decoder_targets = decoder_targets[indices]

split = int(len(encoder_inputs) * 0.9)

train_ds = tf.data.Dataset.from_tensor_slices((
    {'encoder_input': encoder_inputs[:split], 'decoder_input': decoder_inputs[:split]},
    decoder_targets[:split]
)).shuffle(10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((
    {'encoder_input': encoder_inputs[split:], 'decoder_input': decoder_inputs[split:]},
    decoder_targets[split:]
)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(f"✓ Train: {split:,}, Val: {len(encoder_inputs)-split:,}")

## 5. Build Optimized Model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, GRU, Dense, Dropout,
    Bidirectional, Attention, Concatenate, LayerNormalization
)

print("Building optimized Seq2Seq...")

# Shared embedding (weight sharing saves params)
embedding = Embedding(vocab_size, EMBEDDING_DIM, name='shared_embedding')

# Encoder
encoder_input = Input(shape=(MAX_SEQ_LENGTH,), name='encoder_input')
encoder_embed = embedding(encoder_input)
encoder_embed = Dropout(DROPOUT_RATE)(encoder_embed)
encoder_outputs = Bidirectional(
    GRU(GRU_UNITS, return_sequences=True, dropout=DROPOUT_RATE, recurrent_dropout=0.1),
    name='encoder'
)(encoder_embed)

# Decoder
decoder_input = Input(shape=(MAX_SEQ_LENGTH,), name='decoder_input')
decoder_embed = embedding(decoder_input)  # Weight sharing!
decoder_embed = Dropout(DROPOUT_RATE)(decoder_embed)
decoder_outputs = GRU(
    GRU_UNITS * 2,
    return_sequences=True,
    dropout=DROPOUT_RATE,
    recurrent_dropout=0.1,
    name='decoder'
)(decoder_embed)

# Luong Attention
context = Attention(use_scale=True, name='attention')([decoder_outputs, encoder_outputs])

# Combine
combined = Concatenate()([decoder_outputs, context])
combined = LayerNormalization()(combined)
combined = Dropout(DROPOUT_RATE)(combined)

# Output
output = Dense(vocab_size, activation='softmax', name='output')(combined)

model = Model(
    inputs=[encoder_input, decoder_input],
    outputs=output,
    name='kana_kanji_optimized'
)

# Custom loss with label smoothing
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False,
    ignore_class=char_to_idx['<PAD>']  # Ignore padding
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, clipnorm=1.0),
    loss=loss_fn,
    metrics=['accuracy']
)

model.summary()
params = model.count_params()
print(f"\n✓ Parameters: {params:,}")
print(f"✓ Keras size: ~{params * 4 / 1024 / 1024:.1f} MB")
print(f"✓ TFLite FP16: ~{params * 2 / 1024 / 1024:.1f} MB")

## 6. Train with LR Schedule

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler

# Warmup + Cosine Decay schedule
def lr_schedule(epoch):
    if epoch < WARMUP_EPOCHS:
        # Warmup: gradually increase LR
        return 1e-4 + (1e-3 - 1e-4) * (epoch / WARMUP_EPOCHS)
    else:
        # Cosine decay
        progress = (epoch - WARMUP_EPOCHS) / (NUM_EPOCHS - WARMUP_EPOCHS)
        return 1e-5 + 0.5 * (1e-3 - 1e-5) * (1 + np.cos(np.pi * progress))

callbacks = [
    ModelCheckpoint(f'{MODEL_DIR}/best.keras', monitor='val_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
    LearningRateScheduler(lr_schedule, verbose=1)
]

history = model.fit(
    train_ds,
    epochs=NUM_EPOCHS,
    validation_data=val_ds,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
axes[0].plot(history.history['loss'], label='Train')
axes[0].plot(history.history['val_loss'], label='Val')
axes[0].set_title('Loss'); axes[0].legend()

axes[1].plot(history.history['accuracy'], label='Train')
axes[1].plot(history.history['val_accuracy'], label='Val')
axes[1].set_title('Accuracy'); axes[1].legend()

axes[2].plot(history.history['lr'])
axes[2].set_title('Learning Rate')

plt.savefig(f'{MODEL_DIR}/training.png')
plt.show()

print(f"\n✓ Final Val Accuracy: {max(history.history['val_accuracy'])*100:.2f}%")

## 7. Save & Export

In [ ]:
import json

model.save(f'{MODEL_DIR}/model.keras')

with open(f'{MODEL_DIR}/char_to_idx.json', 'w', encoding='utf-8') as f:
    json.dump(char_to_idx, f, ensure_ascii=False)

with open(f'{MODEL_DIR}/idx_to_char.json', 'w', encoding='utf-8') as f:
    json.dump({str(k): v for k, v in idx_to_char.items()}, f, ensure_ascii=False)

config = {
    'vocab_size': vocab_size,
    'max_seq_length': MAX_SEQ_LENGTH,
    'embedding_dim': EMBEDDING_DIM,
    'gru_units': GRU_UNITS,
    'task': 'kana_to_kanji'
}
with open(f'{MODEL_DIR}/config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("✓ Saved model and config")

In [ ]:
print("Exporting TFLite...")

try:
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
    converter._experimental_lower_tensor_list_ops = False
    
    tflite_model = converter.convert()
    with open(f'{MODEL_DIR}/model.tflite', 'wb') as f:
        f.write(tflite_model)
    print(f"✓ model.tflite ({len(tflite_model)/(1024*1024):.2f}MB)")
    
    # FP16 quantization
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    tflite_fp16 = converter.convert()
    with open(f'{MODEL_DIR}/model_fp16.tflite', 'wb') as f:
        f.write(tflite_fp16)
    print(f"✓ model_fp16.tflite ({len(tflite_fp16)/(1024*1024):.2f}MB)")
except Exception as e:
    print(f"⚠ Error: {e}")

## 8. Verification

In [ ]:
print("="*60)
print("VERIFICATION")
print("="*60)

def convert(kana_text, max_len=25):
    enc_input = np.array([encode_seq(kana_text, MAX_SEQ_LENGTH)])
    dec_input = np.zeros((1, MAX_SEQ_LENGTH), dtype=np.int32)
    dec_input[0, 0] = char_to_idx['<BOS>']
    
    result = []
    for i in range(max_len):
        preds = model.predict({'encoder_input': enc_input, 'decoder_input': dec_input}, verbose=0)
        next_idx = np.argmax(preds[0, i])
        next_char = idx_to_char.get(next_idx, '<UNK>')
        if next_char == '<EOS>':
            break
        if next_char not in SPECIAL_TOKENS:
            result.append(next_char)
        if i + 1 < MAX_SEQ_LENGTH:
            dec_input[0, i + 1] = next_idx
    return ''.join(result)

# Test with HIRAGANA
tests = [
    ('ありがとう', '有難う'),
    ('ございます', 'ございます'),
    ('おせわ', 'お世話'),
    ('しんじゅく', '新宿'),
    ('とうきょう', '東京'),
    ('にほん', '日本'),
    ('こんにちは', '今日は'),
    ('おねがいします', 'お願いします'),
]

print("\n📝 Results:")
correct = 0
for kana, expected in tests:
    result = convert(kana)
    ok = '✓' if result == expected else '✗'
    if result == expected:
        correct += 1
    print(f"  {ok} {kana} → {result} (expected: {expected})")

print(f"\n✅ {correct}/{len(tests)} correct ({correct/len(tests)*100:.0f}%)")

In [ ]:
print("\n📦 Files:")
for f in sorted(os.listdir(MODEL_DIR)):
    path = f'{MODEL_DIR}/{f}'
    if os.path.isfile(path):
        size = os.path.getsize(path)
        unit = 'MB' if size > 1024*1024 else 'KB'
        size_str = f"{size/(1024*1024):.2f}" if size > 1024*1024 else f"{size/1024:.1f}"
        print(f"  {f}: {size_str} {unit}")